In [0]:
a = []
while(1):
    a.append('1')

In [0]:
from google.colab import files 
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
import io 
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['train.csv'])) 
print(type(data))
print(data.head())

<class 'pandas.core.frame.DataFrame'>
   id  ... label
0   0  ...     1
1   1  ...     0
2   2  ...     1
3   3  ...     1
4   4  ...     1

[5 rows x 5 columns]


In [0]:
from google.colab import files 
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
import io 
import pandas as pd
datatest = pd.read_csv(io.BytesIO(uploaded['test.csv'])) 
print(type(datatest))
print(datatest.head)

<class 'pandas.core.frame.DataFrame'>
<bound method NDFrame.head of          id  ...                                               text
0     20800  ...  PALO ALTO, Calif.  —   After years of scorning...
1     20801  ...  Russian warships ready to strike terrorists ne...
2     20802  ...  Videos #NoDAPL: Native American Leaders Vow to...
3     20803  ...  If at first you don’t succeed, try a different...
4     20804  ...  42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...     ...  ...                                                ...
5195  25995  ...  Of all the dysfunctions that plague the world’...
5196  25996  ...  WASHINGTON  —   Gov. John Kasich of Ohio on Tu...
5197  25997  ...  Good morning. (Want to get California Today by...
5198  25998  ...  « Previous - Next » 300 US Marines To Be Deplo...
5199  25999  ...  Perhaps you’ve seen the new TV series whose pi...

[5200 rows x 4 columns]>


In [0]:
from google.colab import files 
uploaded = files.upload()

Saving submit.csv to submit.csv


In [0]:
import io 
import pandas as pd
datasubmit = pd.read_csv(io.BytesIO(uploaded['submit.csv'])) 
print(datasubmit.head())

      id  label
0  20800      0
1  20801      1
2  20802      0
3  20803      1
4  20804      1


In [0]:
testdata=pd.merge(datatest,datasubmit,on='id')
testdata.head()


,id,title,author,text,label
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",0
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,0
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",1
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,1


In [0]:
import pandas as pd
finalset=pd.concat([data,testdata])
finalset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
finalset.shape

(26000, 5)

In [0]:
lastset=finalset.dropna()
print(lastset.shape)
print(lastset.head())

(22860, 5)
   id  ... label
0   0  ...     1
1   1  ...     0
2   2  ...     1
3   3  ...     1
4   4  ...     1

[5 rows x 5 columns]


In [0]:
lastset.drop(["id"],axis=1,inplace=True)
lastset.head()


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
lastset.shape

(22860, 4)

In [0]:
lastset_x=lastset.iloc[:,:3]
lastset_x.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [0]:
lastset_y=lastset.iloc[:,3:]
lastset_y.head()

,label
0,1
1,0
2,1
3,1
4,1


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier


In [0]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y


In [0]:
import numpy as np
a=np.array([])
a.size

0

In [0]:
def passive_aggressive(train_x,train_y,test_x,test_y,stop=False):
    
    if stop==True:
        print("Early Stopping is being used")
    
    pac= PassiveAggressiveClassifier(early_stopping=stop)
    pac.fit(train_x, train_y)
    predicted_pac_train=pac.predict(train_x)
    predicted_pac_test= pac.predict(test_x)
    
    score_train= metrics.accuracy_score(train_y, predicted_pac_train)
    score_test=metrics.accuracy_score(test_y,predicted_pac_test)
    
    print("train accuracy:",score_train)
    cm_pac_train= metrics.confusion_matrix(train_y, predicted_pac_train)
    print("train confusion matrix")
    print(cm_pac_train)
    print("test accuracy:",score_test)
    cm_pac_test= metrics.confusion_matrix(test_y, predicted_pac_test)
    print("test confusion matrix")
    print(cm_pac_test)


In [0]:
#mult_tfidf start
def mult_tfidf_pac(x1,y,stop=False):
    from sklearn.feature_extraction.text import TfidfVectorizer

    
    train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        #train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



    tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
    tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
    tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        #tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        #tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 

        #train=pd.concat([tfidf_train_x1,tfidf_train_x2],axis=1)
        #test=pd.concat([tfidf_test_x1,tfidf_test_x2],axis=1)
    print(tfidf_train_x1.shape)
    print(tfidf_test_x1.shape)
    #print(train_y2.shape)
    #print(test_y2.shape)

    passive_aggressive(tfidf_train_x1,train_y1,tfidf_test_x1,test_y1,stop)
    
    '''else:
        train_x1,train_y1,test_x1,test_y1=an.split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=an.split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=an.split_data(x3,y)
        
        tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=0.02)   # a TFIDF vectorizer
        tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
        tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

        tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
        tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 
        
        tfidf_train_x3 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x3).todense())  #fitting the training data
        tfidf_test_x3 = pd.DataFrame(tfidf_vectorizer.transform(test_x3).todense())
        
        train=pd.concat([tfidf_train_x1,tfidf_train_x2,tfidf_train_x3],axis=1)
        test=pd.concat([tfidf_test_x1,tfidf_test_x2,tfidf_test_x3],axis=1)
        
        passive_aggressive(train,train_y2,test,test_y2,stop)'''

In [0]:
 mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=lastset_x.text,stop=True)

"""
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Early Stopping is being used
train accuracy: 0.8922244094488189
train confusion matrix
[[8744 1273]
 [ 698 7573]]
test accuracy: 0.8388013998250219
test confusion matrix
[[2088  469]
 [ 268 1747]]
 """

In [0]:
 mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=lastset_x.text,stop=False)
"""
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
train accuracy: 0.8986767279090113
train confusion matrix
[[8854 1163]
 [ 690 7581]]
test accuracy: 0.8261154855643045
test confusion matrix
[[2081  476]
 [ 319 1696]]
 """

In [0]:
mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 16)
(4572, 16)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.8594706911636045
train confusion matrix
[[8162 1855]
 [ 715 7556]]
test accuracy: 0.8514873140857393
test confusion matrix
[[2063  494]
 [ 185 1830]]


In [0]:
mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 16)
(4572, 16)
(18288,)
(4572,)
train accuracy: 0.7832458442694663
train confusion matrix
[[8723 1294]
 [2670 5601]]
test accuracy: 0.776246719160105
test confusion matrix
[[2192  365]
 [ 658 1357]]


In [0]:
mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 2777)
(4572, 2777)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.8448162729658792
train confusion matrix
[[7475 2542]
 [ 296 7975]]
test accuracy: 0.8094925634295713
test confusion matrix
[[1827  730]
 [ 141 1874]]


In [0]:
mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 2777)
(4572, 2777)
(18288,)
(4572,)
train accuracy: 0.9116360454943132
train confusion matrix
[[9501  516]
 [1100 7171]]
test accuracy: 0.8543307086614174
test confusion matrix
[[2286  271]
 [ 395 1620]]


In [0]:
mult_tfidf_pac(x1=lastset_x.author,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 2767)
(4572, 2767)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.8787182852143482
train confusion matrix
[[8299 1718]
 [ 500 7771]]
test accuracy: 0.8184601924759405
test confusion matrix
[[1962  595]
 [ 235 1780]]


In [0]:
mult_tfidf_pac(x1=lastset_x.author,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 2767)
(4572, 2767)
(18288,)
(4572,)
train accuracy: 0.8821084864391951
train confusion matrix
[[9662  355]
 [1801 6470]]
test accuracy: 0.8278652668416447
test confusion matrix
[[2323  234]
 [ 553 1462]]


In [0]:
mult_tfidf_pac(x1=lastset_x.title,y=lastset_y.label,stop=False)

Final size of train/test : 18288 / 4572
(18288, 13)
(4572, 13)
train accuracy: 0.7554133858267716
train confusion matrix
[[8604 1413]
 [3060 5211]]
test accuracy: 0.7456255468066492
test confusion matrix
[[2163  394]
 [ 769 1246]]


In [0]:
mult_tfidf_pac(x1=lastset_x.author,y=lastset_y.label,stop=False)

Final size of train/test : 18288 / 4572
(18288, 3)
(4572, 3)
train accuracy: 0.4675196850393701
train confusion matrix
[[ 380 9637]
 [ 101 8170]]
test accuracy: 0.45691163604549434
test confusion matrix
[[  98 2459]
 [  24 1991]]


In [0]:
mult_tfidf_pac(x1=lastset_x.text,y=lastset_y.label,stop=False)

Final size of train/test : 18288 / 4572
(18288, 2764)
(4572, 2764)
train accuracy: 0.9029965004374453
train confusion matrix
[[9411  606]
 [1168 7103]]
test accuracy: 0.8342082239720034
test confusion matrix
[[2236  321]
 [ 437 1578]]


In [0]:
mult_tfidf_pac(x1=lastset_x.title,y=lastset_y.label,stop=True)

Final size of train/test : 18288 / 4572
(18288, 13)
(4572, 13)
Early Stopping is being used
train accuracy: 0.8484798775153106
train confusion matrix
[[8064 1953]
 [ 818 7453]]
test accuracy: 0.8388013998250219
test confusion matrix
[[2037  520]
 [ 217 1798]]


In [0]:
mult_tfidf_pac(x1=lastset_x.author,y=lastset_y.label,stop=True)

Final size of train/test : 18288 / 4572
(18288, 3)
(4572, 3)
Early Stopping is being used
train accuracy: 0.4816272965879265
train confusion matrix
[[ 708 9309]
 [ 171 8100]]
test accuracy: 0.4680664916885389
test confusion matrix
[[ 167 2390]
 [  42 1973]]


In [0]:
mult_tfidf_pac(x1=lastset_x.text,y=lastset_y.label,stop=True)

Final size of train/test : 18288 / 4572
(18288, 2764)
(4572, 2764)
Early Stopping is being used
train accuracy: 0.885990813648294
train confusion matrix
[[8468 1549]
 [ 536 7735]]
test accuracy: 0.8278652668416447
test confusion matrix
[[2006  551]
 [ 236 1779]]


In [0]:
#multi_hash start 
def multi_hashv_pac(x1,x2,y,x3=a,stop=False):
    from sklearn.feature_extraction.text import HashingVectorizer
    # Not sure about this, collisions may occur as I forced absolute cause naivie bayes is a bitch and n_features over 
                                                                                        #2**15 causes memory error
    
    if x3.size==0:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)



        hash_vectorizer = HashingVectorizer(stop_words='english',n_features = 2**8)   # a HASH vectorizer
        hash_train_x1 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x1).todense()))  #fitting the training data
        hash_test_x1 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x1).todense()))

        hash_train_x2 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x2).todense()))  #fitting the training data
        hash_test_x2 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x2).todense())) 

        train=pd.concat([hash_train_x1,hash_train_x2],axis=1)
        test=pd.concat([hash_test_x1,hash_test_x2],axis=1)
        print(train.shape)
        print(test.shape)
        print(train_y2.shape)
        print(test_y2.shape)

        passive_aggressive(train,train_y2,test,test_y2,stop)
    
    else:
        train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
        train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
        train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
        hash_vectorizer = HashingVectorizer(stop_words='english',n_features = 2**8)   # a HASH vectorizer
        hash_train_x1 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x1).todense()) ) #fitting the training data
        hash_test_x1 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x1).todense()))

        hash_train_x2 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x2).todense()) ) #fitting the training data
        hash_test_x2 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x2).todense()) )
        
        hash_train_x3 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x3).todense()))  #fitting the training data
        hash_test_x3 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x3).todense()))
        
        train=pd.concat([hash_train_x1,hash_train_x2,hash_train_x3],axis=1)
        test=pd.concat([hash_test_x1,hash_test_x2,hash_test_x3],axis=1)
        
        passive_aggressiv(train,train_y2,test,test_y2,stop)

In [0]:
" " " multi_hashv_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=lastset_x.text,stop=False)
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
train accuracy: 0.849682852143482
train confusion matrix
[[9172  845]
 [1904 6367]]
test accuracy: 0.8285214348206474
test confusion matrix
[[2304  253]
 [ 531 1484]]

In [0]:
" " " mult_tfidf_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=lastset_x.text,stop=True)
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
Early Stopping is being used
train accuracy: 0.8626968503937008
train confusion matrix
[[9417  600]
 [1911 6360]]
test accuracy: 0.8248031496062992
test confusion matrix
[[2310  247]
 [ 554 1461]]

In [0]:
multi_hashv_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.8269903762029747
train confusion matrix
[[7573 2444]
 [ 720 7551]]
test accuracy: 0.8191163604549432
test confusion matrix
[[1927  630]
 [ 197 1818]]


In [0]:
multi_hashv_pac(x1=lastset_x.title,x2=lastset_x.author,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
train accuracy: 0.838746719160105
train confusion matrix
[[9074  943]
 [2006 6265]]
test accuracy: 0.8230533683289589
test confusion matrix
[[2265  292]
 [ 517 1498]]


In [0]:
multi_hashv_pac(x1=lastset_x.title,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
train accuracy: 0.8337160979877515
train confusion matrix
[[8244 1773]
 [1268 7003]]
test accuracy: 0.8143044619422573
test confusion matrix
[[2045  512]
 [ 337 1678]]


In [0]:
multi_hashv_pac(x1=lastset_x.title,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.8354658792650919
train confusion matrix
[[8379 1638]
 [1371 6900]]
test accuracy: 0.8125546806649169
test confusion matrix
[[2097  460]
 [ 397 1618]]


In [0]:
multi_hashv_pac(x1=lastset_x.author,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=False)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
train accuracy: 0.811078302712161
train confusion matrix
[[8371 1646]
 [1809 6462]]
test accuracy: 0.7944006999125109
test confusion matrix
[[2097  460]
 [ 480 1535]]


In [0]:
multi_hashv_pac(x1=lastset_x.author,x2=lastset_x.text,y=lastset_y.label,x3=a,stop=True)

Final size of train/test : 18288 / 4572
Final size of train/test : 18288 / 4572
(18288, 512)
(4572, 512)
(18288,)
(4572,)
Early Stopping is being used
train accuracy: 0.806758530183727
train confusion matrix
[[8720 1297]
 [2237 6034]]
test accuracy: 0.7935258092738408
test confusion matrix
[[2193  364]
 [ 580 1435]]
